# **Parameter Optimisation for GITT** 

This notebook is created to provide an example for ```GITT``` optimisation problem. Diffisuion Coefficient is optimised in this notebook. Only voltage will be considered for optimisation in this example.
 The workflow is as follows:


In [1]:
#%pip install git+https://github.com/paramm-team/pybamm-param.git -q
import pybamm
import pbparam
import pandas as pd
import os
os.chdir(os.path.join(pbparam.__path__[0], "input", "data"))

We are creating a syntethic data first.

In [2]:
syn_model = pybamm.lithium_ion.SPM()

In [3]:
syn_param = syn_model.default_parameter_values

syn_param.update(
            {
                "Reference OCP [V]": 4.2,
                "Derivative of the OCP wrt stoichiometry [V]": -1,
                "Contact resistance [Ohm]": 0.1,
                "Positive electrode diffusivity [m2.s-1]": 4e-15
                },
            check_already_exists=False,
        )

syn_sim = pybamm.Simulation(syn_model, parameter_values=syn_param)
syn_solution = syn_sim.solve([0, 100])

In [4]:
d = {'Time [s]':syn_solution["Time [s]"].entries, 'Voltage [V]': syn_solution["Voltage [V]"].entries}

data = pd.DataFrame(data = d)

PbParam model to be used for parameter optimisation needs to be defined:

In [5]:
model = pbparam.WeppnerHuggins()
#model = pbparam.BasicGITT()

Rest of the PyBaMM simulation requirements and initial parameters defined here:

In [6]:
param = model.default_parameter_values # Initial parameters to be used.

# Update some parameters to match the experimental setup
param.update(
    {
        "Reference OCP [V]": 4.2,
        "Derivative of the OCP wrt stoichiometry [V]": -1,
    },
    check_already_exists=False,
)

simulation = pybamm.Simulation(model, parameter_values=param)

In [7]:
# optimisation problem is GITT.
opt = pbparam.GITT(simulation=simulation, data=data)

Then, Optimisation method should be defined in this step. DiferentialEvolution(https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html) or ScipyMinimize(https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)

can be used to minimise cost function. For faster results, ScipyMinimize with ```"Nelder-Mead"``` method can be used. However, ScipyDifferentialEvolution is more robust and has provided lower value for most of the cases. 

In [8]:
# optimiser = pbparam.ScipyDifferentialEvolution(
    # extra_options={"workers": 4, "polish": True, "updating": "deferred", "disp": True}
# )
optimiser = pbparam.ScipyMinimize(method="Nelder-Mead")

After providing everything for optimisation algorithm, it can be performed using `optimise(optimisation_problem)`

In [9]:
result = optimiser.optimise(opt)
# optimised values of parameters and function values can be printed as below.
print(result)

TypeError: Cannot process parameter '<function lico2_diffusivity_Dualfoil1998 at 0x15f4a6480>'

```plot()``` function can be used to present result graphically.

In [ ]:
result.plot()

AttributeError: 'GITT' object has no attribute 'simulation'